In [1]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

In [2]:
data = pd.read_excel('Llamados2024.xlsx')

1. Administrativos v/s Cantidad de pacientes contactados
• En el estado inicial, sin filtros asignados, deberá mostrar el conteo ordenado de menor a mayor.
• Este gráfico deberá permitir seleccionar, respecto a la cantidad, entre 3 categorías: Entre 0 a 4000, Entre
4001 a 8000 y Entre 8001 en adelante.
• Adicionalmente deberá existir un control que permitirá seleccionar el nombre de un administrativo, el que al
activarse hará una comparación de él frente al promedio.

2. Médicos v/s Horas Anuladas
• En el estado inicial, sin filtros asignados, deberá mostrar el conteo de los promedios de cada médico,
ordenado de menor a mayor.
• Los controles por considerar son: Listado de Médicos (incluir un ítem “todos”), Dado Por (Web, Médico o
Administrativo) y Meses. El control de Dado Por deberá filtrar la información de horas anuladas, pero que
fueron “dadas por” el tipo de personal seleccionado.
• En este gráfico los controles podrán ser “complementarios”, por lo que, por ejemplo, podría visualizar el
listado de las horas anuladas que fueron dadas por un Administrativo en el mes de Julio.


In [5]:
# Filter data to get the count of patients contacted by each administrative
contacted_counts = data['contactadopor'].value_counts().reset_index()
contacted_counts.columns = ['contactadopor', 'count']
contacted_counts = contacted_counts.sort_values(by='count')


In [7]:
# Initialize the Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Administrativos v/s Cantidad de pacientes contactados"),
    dcc.Dropdown(
        id='category-dropdown',
        options=[
            {'label': 'Sin filtro', 'value': 'all'},
            {'label': 'Entre 0 a 4000', 'value': '0-4000'},
            {'label': 'Entre 4001 a 8000', 'value': '4001-8000'},
            {'label': 'Entre 8001 en adelante', 'value': '8001+'}
        ],
        value='all'
    ),
    dcc.Dropdown(
        id='admin-dropdown',
        options=[{'label': admin, 'value': admin} for admin in contacted_counts['contactadopor']],
        placeholder="Select an administrative"
    ),
    dcc.Graph(id='bar-graph')
])

@app.callback(
    Output('bar-graph', 'figure'),
    [Input('category-dropdown', 'value'),
     Input('admin-dropdown', 'value')]
)
def update_graph(selected_category, selected_admin):
    if selected_category == '0-4000':
        filtered_data = contacted_counts[contacted_counts['count'] <= 4000]
    elif selected_category == '4001-8000':
        filtered_data = contacted_counts[(contacted_counts['count'] > 4000) & (contacted_counts['count'] <= 8000)]
    elif selected_category == '8001+':
        filtered_data = contacted_counts[contacted_counts['count'] > 8000]
    else:
        filtered_data = contacted_counts

    fig = px.bar(filtered_data, x='contactadopor', y='count', title='Cantidad de pacientes contactados')

    if selected_admin:
        admin_count = contacted_counts[contacted_counts['contactadopor'] == selected_admin]['count'].values[0]
        avg_count = contacted_counts['count'].mean()
        fig.add_trace(px.bar(x=[selected_admin, 'Promedio'], y=[admin_count, avg_count], title='Comparación con el promedio').data[0])

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)